In [35]:
import requests
import pandas as pd

In [36]:
import requests

url = "https://api.football-data.org/v4/competitions/PD/matches?season=2023"

headers = {
    "X-Auth-Token": "f3f225de51ce40f3b8e8558377c4f7dd"
}

response = requests.get(url, headers=headers)

In [37]:
data = response.json()

In [38]:
data

{'filters': {'season': 2023},
 'resultSet': {'count': 380,
  'first': '2023-08-11',
  'last': '2024-05-26',
  'played': 380},
 'competition': {'id': 2014,
  'name': 'Primera Division',
  'code': 'PD',
  'type': 'LEAGUE',
  'emblem': 'https://crests.football-data.org/laliga.png'},
 'matches': [{'area': {'id': 2224,
    'name': 'Spain',
    'code': 'ESP',
    'flag': 'https://crests.football-data.org/760.svg'},
   'competition': {'id': 2014,
    'name': 'Primera Division',
    'code': 'PD',
    'type': 'LEAGUE',
    'emblem': 'https://crests.football-data.org/laliga.png'},
   'season': {'id': 1577,
    'startDate': '2023-08-13',
    'endDate': '2024-05-26',
    'currentMatchday': 38,
    'winner': None},
   'id': 438482,
   'utcDate': '2023-08-11T17:30:00Z',
   'status': 'FINISHED',
   'matchday': 1,
   'stage': 'REGULAR_SEASON',
   'group': None,
   'lastUpdated': '2023-10-09T15:20:25Z',
   'homeTeam': {'id': 267,
    'name': 'UD Almería',
    'shortName': 'Almería',
    'tla': 'ALM',
 

In [39]:
matches_data = [{
    'utcDate': match['utcDate'],
    'homeTeam': match['homeTeam']['name'],
    'awayTeam': match['awayTeam']['name'],
    'score_home': match['score']['fullTime']['home'],
    'score_away': match['score']['fullTime']['away'],
    'matchday': match['matchday'],
    'stage': match['stage']
} for match in data['matches']]

df = pd.DataFrame(matches_data)

In [40]:
import pandas as pd

# Initialize Elo ratings for each team
initial_elo = 1500
team_elo = {}

# Ensure all teams are initialized
all_teams = set(df['homeTeam']).union(set(df['awayTeam']))
for team in all_teams:
    team_elo[team] = initial_elo

# Sort matches chronologically if a date column exists
if 'date' in df.columns:
    df = df.sort_values('date')

# Function to calculate the expected score
def expected_score(rating_a, rating_b):
    return 1 / (1 + 10 ** ((rating_b - rating_a) / 400))

# Function to update Elo ratings
def update_elo(elo_a, elo_b, score_a, score_b, k=30):
    expected_a = expected_score(elo_a, elo_b)
    expected_b = expected_score(elo_b, elo_a)

    # Update the Elo ratings based on the actual score
    new_elo_a = elo_a + k * (score_a - expected_a)
    new_elo_b = elo_b + k * (score_b - expected_b)

    return new_elo_a, new_elo_b

# Store results in a list
elo_history = []

# Iterate over each match
for idx, row in df.iterrows():
    home_team = row['homeTeam']
    away_team = row['awayTeam']
    home_score = row['score_home']
    away_score = row['score_away']
    
    # Get current Elo ratings
    home_elo = team_elo.get(home_team, initial_elo)
    away_elo = team_elo.get(away_team, initial_elo)
    
    # Determine match outcome (1 for win, 0.5 for draw, 0 for loss)
    if home_score > away_score:
        home_actual = 1
        away_actual = 0
    elif home_score < away_score:
        home_actual = 0
        away_actual = 1
    else:
        home_actual = 0.5
        away_actual = 0.5
    
    # Update Elo ratings
    new_home_elo, new_away_elo = update_elo(home_elo, away_elo, home_actual, away_actual)
    
    # Store updated Elo ratings
    team_elo[home_team] = new_home_elo
    team_elo[away_team] = new_away_elo
    
    # Track Elo history for analysis or display
    elo_history.append({
        'date': row.get('date', None),  # Store date if available
        'home_team': home_team,
        'away_team': away_team,
        'home_elo_before': home_elo,
        'away_elo_before': away_elo,
        'home_elo_after': new_home_elo,
        'away_elo_after': new_away_elo,
        'home_score': home_score,
        'away_score': away_score
    })

# Convert Elo history to a DataFrame for analysis or display
elo_df = pd.DataFrame(elo_history)

# Remove matches that haven't been played (NaN scores)
elo_df = elo_df.dropna(subset=['home_score', 'away_score'])

elo_df

,date,home_team,away_team,home_elo_before,away_elo_before,home_elo_after,away_elo_after,home_score,away_score
0,None,UD Almería,Rayo Vallecano de Madrid,1500.000000,1500.000000,1485.000000,1515.000000,0,2
1,None,Sevilla FC,Valencia CF,1500.000000,1500.000000,1485.000000,1515.000000,1,2
2,None,Real Sociedad de Fútbol,Girona FC,1500.000000,1500.000000,1500.000000,1500.000000,1,1
3,None,UD Las Palmas,RCD Mallorca,1500.000000,1500.000000,1500.000000,1500.000000,1,1
4,None,Athletic Club,Real Madrid CF,1500.000000,1500.000000,1485.000000,1515.000000,0,2
...,...,...,...,...,...,...,...,...,...
375,None,Real Madrid CF,Real Betis Balompié,1728.464390,1523.924174,1720.529838,1531.858726,0,0
376,None,Getafe CF,RCD Mallorca,1439.172352,1448.760691,1424.586209,1463.346834,1,2
377,None,RC Celta de Vigo,Valencia CF,1474.293505,1465.535573,1473.915474,1465.913603,2,2
378,None,UD Las Palmas,Deportivo Alavés,1408.949207,1495.474542,1412.609447,1491.814302,1,1


In [41]:
# Get the latest Elo ratings for each team
latest_home_elos = elo_df.groupby("home_team")["home_elo_after"].last().reset_index()
latest_away_elos = elo_df.groupby("away_team")["away_elo_after"].last().reset_index()

# Merge the two (some teams may only appear in home or away)
standings = pd.merge(latest_home_elos, latest_away_elos, 
                     left_on="home_team", right_on="away_team", 
                     how="outer")

# Fill missing values where a team may have only played home or away
standings["team"] = standings["home_team"].combine_first(standings["away_team"])
standings["home_elo"] = standings["home_elo_after"].fillna(initial_elo)
standings["away_elo"] = standings["away_elo_after"].fillna(initial_elo)

# Keep only relevant columns
standings = standings[["team", "home_elo", "away_elo"]]

# Compute the overall Elo rating (average of home and away)
standings["overall_elo"] = standings[["home_elo", "away_elo"]].mean(axis=1)

# Sort standings by highest Elo rating
standings = standings.sort_values(by="overall_elo", ascending=False).reset_index(drop=True)

# Display standings
standings[['team', 'overall_elo']]

,team,overall_elo
0,Real Madrid CF,1724.497114
1,FC Barcelona,1663.136855
2,Girona FC,1610.329468
3,Club Atlético de Madrid,1596.132133
4,Athletic Club,1566.747602
5,Villarreal CF,1553.165197
6,Real Sociedad de Fútbol,1542.826872
7,Real Betis Balompié,1527.891450
8,Deportivo Alavés,1493.644422
9,RC Celta de Vigo,1474.104490
